# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

s = 0
for sqrt_value in generator:
    s += sqrt_value
    print(sqrt_value)

print("summ", s)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
summ 32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
#Install the dependencies
#pip install dlt[duckdb]

     ---------------------------------------- 0.0/50.6 kB ? eta -:--:--
     -------------------------------- ------- 41.0/50.6 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 50.6/50.6 kB 858.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/49.6 kB ? eta -:--:--
     ---------------------------------------- 49.6/49.6 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.3 MB 8.6 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/10.3 MB 11.0 MB/s eta 0:00:01
   ------ --------------------------------- 1.6/10.3 MB 12.4 MB/s eta 0:00:01
   --------- ------------------------------ 2.4/10.3 MB 14.0 MB/s eta 0:00:01
   ------------ --------------------------- 3.2/10.3 MB 14.8 MB/s eta 0:00:01
   ---------------- ----------------------- 4.2/10.3 MB 15.7 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.3 MB 16.6 MB/s eta 0:00:01
 


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\mbary\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [27]:
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

ppl_1 = generators_pipeline.run(people_1(),
										table_name="people_1",
                                        primary_key="ID",
										write_disposition="replace")



In [22]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")

rides = conn.sql("SELECT SUM(age) FROM people_1").df()
display(rides)


,sum(age)
0,140.0


In [29]:
ppl_2 = generators_pipeline.run(people_2(),
										table_name="people_1",
                                        primary_key="ID",
										write_disposition="merge")

In [28]:
rides = conn.sql("SELECT * FROM people_1").df()
display(rides)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707234971.1677208,tGwpuljTNRNGTg,None
1,2,Person_2,27,City_A,1707234971.1677208,RdJKTUHXbBqnHg,None
2,3,Person_3,28,City_A,1707234971.1677208,M05xa8psRUiAqA,None
3,4,Person_4,29,City_A,1707234971.1677208,PL66GrhE7PMh4w,None
4,5,Person_5,30,City_A,1707234971.1677208,zRE9SjT1b60rmg,None


In [30]:
rides = conn.sql("SELECT * FROM people_1").df()
display(rides)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707234971.1677208,tGwpuljTNRNGTg,None
1,2,Person_2,27,City_A,1707234971.1677208,RdJKTUHXbBqnHg,None
2,5,Person_5,35,City_B,1707235039.430098,V8TFBTltYbnRew,Job_5
3,4,Person_4,34,City_B,1707235039.430098,gO0mv7K/L7Vfjg,Job_4
4,3,Person_3,33,City_B,1707235039.430098,kowuOtlwqp8Rsg,Job_3
5,6,Person_6,36,City_B,1707235039.430098,7PmK98HD7fxpjw,Job_6
6,7,Person_7,37,City_B,1707235039.430098,wXIuVtH8VbidIw,Job_7
7,8,Person_8,38,City_B,1707235039.430098,tWW4xPSHMm3TYQ,Job_8


In [31]:
rides = conn.sql("SELECT SUM(age) FROM people_1").df()
display(rides)

,sum(age)
0,266.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX